## Particle-Based Surface Shape Modeling
## PART 1 - Open Images and create Meshes

```
conda activate shapeworks
```



In [12]:
import shapeworks as sw

In [13]:
# IMPORTS

import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from pathlib import Path
from scipy.ndimage import zoom
from scipy.spatial import cKDTree
import pyvista as pv
import vtk

### 1. Load shape images

In [14]:
# PATHS

data_path="./DATA/"
shapeExtention = '.nii.gz'
shapeFilenames = sorted(glob.glob(data_path + '*' + shapeExtention)) 

print ('Number of shapes: ' + str(len(shapeFilenames)))
for shapeFilename in shapeFilenames:
    shapeFilename = Path(shapeFilename)
    print(shapeFilename)


Number of shapes: 2
DATA\F006_label_4_RF.nii.gz
DATA\F099_label_4_RF.nii.gz


In [15]:
# LOAD AND DOWNSAMPLE SEGMENTATIONS

def downsample_volume(volume, factor):
    data = volume.toArray()
    zoom_factors = (1 / factor, 1 / factor, 1 / factor)
    downsampled_data = zoom(data, zoom_factors, order=1)  # Using linear interpolation
    return sw.Image(downsampled_data.astype(np.float32))

# Adjust the downsample factor based on your hardware capabilities
downsample_factor = 2

# List of shape segmentations
small_shapes = []

# List of shape names (shape files prefixes) to be used for saving outputs and visualizations
small_shapeNames = [] 

# Loop over all shape files and load individual segmentations
for shapeFilename in shapeFilenames:
    print('Loading: ' + str(shapeFilename))
    
    # Convert to Path object to access .name attribute
    shapeFilename = Path(shapeFilename)
    
    # Current shape name
    segFilename = shapeFilename.name
    shapeName = segFilename[:-len(shapeExtention)]
    small_shapeNames.append(shapeName)
    
    # Load segmentation
    shapeSeg = sw.Image(str(shapeFilename))
    
    # Downsample the volume if needed
    downsampled_shapeSeg = downsample_volume(shapeSeg, downsample_factor)
    
    # Append to the shape list
    small_shapes.append(downsampled_shapeSeg)

num_samples = len(small_shapes)
print('\n' + str(num_samples) + ' segmentations are loaded for the dataset...')


Loading: ./DATA\F006_label_4_RF.nii.gz
Loading: ./DATA\F099_label_4_RF.nii.gz

2 segmentations are loaded for the dataset...


### 2. Visualize Images

In [16]:
#VISUALIZATION PARAMETERS

use_same_window = False # plot using multiple rendering windows if false
notebook        = False # True will enable the plots to lie inline
show_borders    = True  # show borders for each rendering window
shade_volumes   = True  # use shading when performing volume rendering
color_map       = "viridis" # color map for volume rendering, e.g., 'bone', 'coolwarm', 'cool', 'viridis', 'magma'
show_axes       = True  # show a vtk axes widget for each rendering window
show_bounds     = True  # show volume bounding box
show_all_edges  = True  # add an unlabeled and unticked box at the boundaries of plot. 
font_size       = 10    # text font size for windows
link_views      = True  # link all rendering windows so that they share same camera and axes boundaries


# EXAMPLE USAGE

""" 
sw.plot_volumes(small_shapes,    
             volumeNames     = small_shapeNames, 
             use_same_window = use_same_window,
             notebook        = notebook,
             show_borders    = show_borders,  
             shade_volumes   = shade_volumes, 
             color_map       = color_map,
             show_axes       = show_axes,  
             show_bounds     = show_bounds,
             show_all_edges  = show_all_edges, 
             font_size       = font_size,   
             link_views      = link_views
             ) 
"""

print()

### 3. Convert images

In [21]:
# Threshold and binarize the segmentation images
for idx in range(len(small_shapes)):
    shapeSeg = small_shapes[idx]
    # Convert to numpy array to inspect voxel values
    shapeSeg_array = shapeSeg.toArray()
    # Get unique voxel values
    voxelValues = np.unique(shapeSeg_array)
    print(f"Shape {idx} voxel values count: {len(voxelValues)}")
    
    # Determine threshold values
    minVal = voxelValues.min()
    maxVal = voxelValues.max()
    threshold = (minVal + maxVal) / 2.0  # Midpoint threshold

    # Binarize the image: set voxels >= threshold to 1.0, else 0.0
    shapeSeg.binarize(minVal=threshold, maxVal=maxVal, innerVal=1.0, outerVal=0.0)
    
    # Verify binarization
    binarized_voxelValues = np.unique(shapeSeg.toArray())
    print(f"Shape {idx} voxel values after binarization: {binarized_voxelValues}")
    
    # Update the image in the list
    small_shapes[idx] = shapeSeg


Shape 0 voxel values count: 2
Shape 0 voxel values after binarization: [0. 1.]
Shape 1 voxel values count: 2
Shape 1 voxel values after binarization: [0. 1.]


In [18]:
# CONVERT TO VTK IMAGE
shapeSeg1 = small_shapes[0]
shapeSeg2 = small_shapes[1]
shapeSeg1_vtk = sw.sw2vtkImage(shapeSeg1, verbose = True)
shapeSeg2_vtk = sw.sw2vtkImage(shapeSeg2, verbose = True)
# sw.plot_volumes(shapeSeg1_vtk)
# sw.plot_volumes(shapeSeg2_vtk)

shapeworks image header information: 
{
	dims: [233, 226, 759],
	origin: [0, 0, 0],
	size: [233, 226, 759],
	spacing: [1, 1, 1]
}

vtk image header information: 
ImageData (0x27ee9ba7d00)
  N Cells:      39567600
  N Points:     39967422
  X Bounds:     0.000e+00, 2.320e+02
  Y Bounds:     0.000e+00, 2.250e+02
  Z Bounds:     0.000e+00, 7.580e+02
  Dimensions:   233, 226, 759
  Spacing:      1.000e+00, 1.000e+00, 1.000e+00
  N Arrays:     1
shapeworks image header information: 
{
	dims: [230, 223, 772],
	origin: [0, 0, 0],
	size: [230, 223, 772],
	spacing: [1, 1, 1]
}

vtk image header information: 
ImageData (0x27eebdd1e80)
  N Cells:      39196098
  N Points:     39595880
  X Bounds:     0.000e+00, 2.290e+02
  Y Bounds:     0.000e+00, 2.220e+02
  Z Bounds:     0.000e+00, 7.710e+02
  Dimensions:   230, 223, 772
  Spacing:      1.000e+00, 1.000e+00, 1.000e+00
  N Arrays:     1


In [19]:
# CONVERT TO MESH

def convert_plot_save(shapeSeg, output_filename):
    print(f'\nCreating mesh: {output_filename}')
    # Convert shapeSeg to numpy array
    shapeSeg_array = shapeSeg.toArray()
    # Get unique voxel values
    voxelValues = np.unique(shapeSeg_array)
    # Suppress scientific notation
    np.set_printoptions(suppress=True)
    print('Voxel values:' + str(voxelValues))
    # Display warning
    if len(voxelValues) > 2:
        print('WARNING: Voxels have more than two distinct values')
        print('PLEASE make sure to use binary segmentations')
    else:
        print('Shape is a binary segmentation')
    # Get min and max values
    minVal = shapeSeg_array.min()
    maxVal = shapeSeg_array.max()
    # Compute isovalue
    isoValue = (maxVal - minVal) / 2.0
    print('isoValue = ' + str(isoValue))
    # Extract isosurface
    shapeMesh = shapeSeg.toMesh(isovalue=isoValue)
    # Convert to VTK format
    shapeMesh_vtk = sw.sw2vtkMesh(shapeMesh)
    # Plot
    sw.plot_meshes([shapeMesh_vtk])
    # Convert VTK mesh to PyVista object
    pv_mesh = pv.wrap(shapeMesh_vtk)
    # Save the VTK mesh
    pv_mesh.save(output_filename)
    print(f'Saved VTK mesh to: {output_filename}')
    
def exportMesh(vtk_mesh, output_filename):
    pv_mesh = pv.wrap(vtk_mesh)
    pv_mesh.save(output_filename)
    print(f'Saved VTK mesh to: {output_filename}')
    
def seg2mesh(shapeSeg):
    print(f'\nCreating mesh...')
    # Convert shapeSeg to numpy array
    shapeSeg_array = shapeSeg.toArray()
    # Get unique voxel values
    voxelValues = np.unique(shapeSeg_array)
    # Suppress scientific notation
    np.set_printoptions(suppress=True)
    print('Voxel values:' + str(voxelValues))
    # Display warning
    if len(voxelValues) > 2:
        print('WARNING: Voxels have more than two distinct values')
    # Get min and max values
    minVal = shapeSeg_array.min()
    maxVal = shapeSeg_array.max()
    # Compute isovalue
    isoValue = (maxVal - minVal) / 2.0
    # Extract isosurface
    shapeMesh = shapeSeg.toMesh(isovalue=isoValue)
    # Convert to VTK format
    shapeMesh_vtk = sw.sw2vtkMesh(shapeMesh, verbose = False)
    return shapeMesh_vtk

# Example usage
# convert_plot_save(shapeSeg1, 'shape1.vtk')
# convert_plot_save(shapeSeg2, 'shape2.vtk')

output_dir = 'OUTPUT'
os.makedirs(output_dir, exist_ok=True)

convert_plot_save(shapeSeg1, output_dir + '/mesh1.vtk')
convert_plot_save(shapeSeg2, output_dir + '/mesh2.vtk')


Creating mesh: OUTPUT/mesh1.vtk
Voxel values:[0. 1.]
Shape is a binary segmentation
isoValue = 0.5
Saved VTK mesh to: OUTPUT/mesh1.vtk

Creating mesh: OUTPUT/mesh2.vtk
Voxel values:[0. 1.]
Shape is a binary segmentation
isoValue = 0.5
Saved VTK mesh to: OUTPUT/mesh2.vtk
